## chainerとは ##

* preferred infrastructureが作ったpythonから利用可能なDeep Learning用ライブラリ
    * https://github.com/pfnet/chainer

## インストール ##

* pipでchainerを入れる

```
$ pip install chainer
```

### CUDA対応にしたい場合は以下の手順を踏んでインストール ###

* CUDAのインストール https://developer.nvidia.com/cuda-downloads?sid=854243
* CuFFTのパッチのダウンロードおよび適用
```
$ cp cufft_patch_mac.tar.gz ~
$ cd /Developer/NVIDIA/CUDA-7.0
$ sudo tar zxvf ~/cufft_patch_mac.tar.gz
```

* 環境変数CUDA_DIR、PATH, LD_LIBRARY_PATHの設定

```
export CUDA_DIR="/usr/local/cuda"
export PATH="${CUDA_DIR}/bin:${PATH}"
export LD_LIBRARY_PATH="${CUDA_DIR}/lib:${LD_LIBRARY_PATH}"
```
    * osx(10.10.3)の場合cudaは/Developer/NVIDIA/CUDA-7.0以下に入るが、同時に/usr/local/cuda以下からシンボリックリンクも張られるため、CUDA_DIRは/usr/local/cudaにする

* 依存しているライブラリのインストール

```
pip install chainer-cuda-deps
```

    * しかしosxでは/Developer/NVIDIA/CUDA-7.0/lib64が存在しないためにリンク時にerrorが発生する
    * 苦肉の策としてlibに対してlib64のシンボリックリンクを張る必要があった   

* pycudaが実行できるかテスト

```python
import pycuda.gpuarray as gpuarray
import pycuda.driver as cuda
import pycuda.autoinit
import numpy as np
a_gpu = gpuarray.to_gpu(np.random.randn(4,4).astype(np.float32))
a_doubled = (2*a_gpu).get()
print a_doubled
print a_gpu
```

* macbook air 2012(mid)だとNVIDIAでなくIntelのGPUなのでautoinitで落ちる
    * 何のために頑張ったのか

## サンプルを実行してみる ##

* https://github.com/pfnet/chainer/tree/master/examples/mnist のmnist_train.pyを試す
    * 複数GPUでパラレルに回すmnist_train_model_parallel.pyも存在

### CPUで試す ###

```sh
$ python train_mnist.py
...
epoch 19
train mean loss=0.0420299975814, accuracy=0.987416676184
test  mean loss=0.0650055166729, accuracy=0.983600005507
epoch 20
train mean loss=0.0456801100779, accuracy=0.986183343232
test  mean loss=0.065424739151, accuracy=0.983800007105
```
と20回試行される。

* Macbook Air 2012(mid)の場合
    * 1.8 GHz Intel Core i5
    * timeコマンドで測ったところ以下の結果(23分)
    
    python train_mnist.py  1415.84s user 147.81s system 123% cpu 21:09.24 total

### GPUで試す ###

```sh
$ python train_mnist.py --gpu=1
```
* osxの場合`export DYLD_LIBRARY_PATH="${CUDA_DIR}/lib:${LD_LIBRARY_PATH}"`しておく必要有
    * しかしNVIDIA GPUでなかったためsegmentation faultが発生し絶望的な気持ちに襲われる

## 中身を読む ##

In [11]:
# 準備
import numpy as np
from chainer import cuda, Variable, FunctionSet, optimizers                                                                                                                       
import chainer.functions  as F 

batchsize = 100                                                                                                                                                                   
n_epoch   = 20                                                                                                                                                                    
n_units   = 1000
N         = 60000

In [12]:
# ニューラルネットワークの構成を決める部分
model = FunctionSet(l1=F.Linear(784, n_units),                                                                                                                                    
                    l2=F.Linear(n_units, n_units),                                                                                                                                
                    l3=F.Linear(n_units, 10)) 

In [17]:
# 前向き計算の定義
# Fはユーティリティクラス, modelはグローバルになっている
# 第三引数trainによって挙動が変わる
def forward(x_data, y_data, train=True):                                                                                                                                          
    x, t = Variable(x_data), Variable(y_data)                                                                                                                                     
    h1 = F.dropout(F.relu(model.l1(x)),  train=train)                                                                                                                             
    h2 = F.dropout(F.relu(model.l2(h1)), train=train)                                                                                                                             
    y  = model.l3(h2)                                                                                                                                                             
    return F.softmax_cross_entropy(y, t), F.accuracy(y, t)

In [14]:
# オプティマイザの設定
# ここでオプティマイザとモデルが紐付けられる？
optimizer = optimizers.Adam()                                                                                                                                                     
optimizer.setup(model.collect_parameters())  

In [15]:
# 各epochの繰り返しにおける学習過程
perm = np.random.permutation(N)
for i in xrange(0, N, batchsize):  # バッチサイズごとにステップ 
    x_batch = x_train[perm[i:i+batchsize]]  # 学習データをバッチサイズに切り出す
    y_batch = y_train[perm[i:i+batchsize]]
    if args.gpu >= 0:
        x_batch = cuda.to_gpu(x_batch)
        y_batch = cuda.to_gpu(y_batch)

    optimizer.zero_grads()
    loss, acc = forward(x_batch, y_batch)
    loss.backward()  # バックプロパゲーション
    optimizer.update()

NameError: name 'x_train' is not defined

In [16]:
# 評価過程, forwardの第三引数がFalseになっている
for i in xrange(0, N_test, batchsize):
    x_batch = x_test[i:i+batchsize]
    y_batch = y_test[i:i+batchsize]
    if args.gpu >= 0:
        x_batch = cuda.to_gpu(x_batch)
        y_batch = cuda.to_gpu(y_batch)

    loss, acc = forward(x_batch, y_batch, train=False)

NameError: name 'N_test' is not defined

## 次回 ##

* 中身をもう少しちゃんと読む
* AWSのCG1インスタンスかG2インスタンスで試す
    * CG1インスタンス
        * GPU: M2050
        * FLOPS: 1.03 Tflops
        * メモリ: 3GB
    * G2インスタンス: GRID K520
        * http://www.nvidia.co.jp/object/cloud-gaming-gpu-boards-jp.html